### data1의 중국어 상품명을 한국어로 번역하고 카테고리를 분류한 결과를 data2에 저장

1. data/categories.txt에 카테고리 목록 저장되어 있음
2. data2.csv
    - product_name_kor : 한국어 상품명 (번역 결과)
    - product_category : 카테고리
    - product_selling_points : 상품 셀링 포인트

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import pandas as pd

In [3]:
llm = ChatOpenAI(model='gpt-3.5-turbo-0125')

prompt_text1 = "Translate '{chinese}' into Korean.(Do not include quotes)"
prompt1 = ChatPromptTemplate.from_template(prompt_text1)

with open("prompts/categories.txt", 'r', encoding='utf-8') as file: categories = file.read()

prompt_text2 = "Classify the product '{product}' into one of the categories below." \
               "(Nothing is needed other than the category itself)\nCategories:\n" + categories
prompt2 = ChatPromptTemplate.from_template(prompt_text2)

prompt_text3 = "Extract a list of 5 selling points from the given brief description of the product in Korean. " \
               "Those 5 selling points should contain 5 various aspects and preferably not overlap with each other. " \
               "The elements of the list should be in one line without any numbering and seperated only by a comma.\n" \
               "Breif description of the product: {product}"
prompt3 = ChatPromptTemplate.from_template(prompt_text3)

parser = StrOutputParser()

chain1 = {"chinese": RunnablePassthrough()} | prompt1 | llm | parser
chain2 = {"product": RunnablePassthrough()} | prompt2 | llm | parser
chain3 = {"product": RunnablePassthrough()} | prompt3 | llm | parser

FileNotFoundError: [Errno 2] No such file or directory: 'data/categories.txt'

In [5]:
product_name_kor = []

final_chain = chain1 | (lambda p: (product_name_kor.append(p), p)[-1]) | chain2

product_name_chn = pd.read_csv("data/data1.csv")["product_name_chn"].tolist()

product_category = final_chain.batch(product_name_chn, config={"max_concurrency": 100})

product_selling_points = chain3.batch(product_name_kor, config={"max_concurrency": 100})

In [6]:
df = pd.DataFrame({
    'product_name_chn': product_name_chn,
    'product_name_kor': product_name_kor,
    'product_category': product_category,
    'product_selling_points': product_selling_points
})

df.to_csv("data/data2.csv", index=False)